In [ ]:
# Modified from https://www.geeksforgeeks.org/multiclass-classification-using-scikit-learn/

import argparse
import os
from azureml.core import Dataset
# importing necessary libraries
import numpy as np

from sklearn import datasets
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

import joblib

from azureml.core.run import Run
run = Run.get_context()


def main():
    parser = argparse.ArgumentParser()

    parser.add_argument('--kernel', type=str, default='linear',
                        help='Kernel type to be used in the algorithm')
    parser.add_argument('--penalty', type=float, default=1.0,
                        help='Penalty parameter of the error term')

    args = parser.parse_args()
    run.log('Kernel type', np.str(args.kernel))
    run.log('Penalty', np.float(args.penalty))

    # loading the iris dataset
    # Getting a dataset

    dataset_name = 'iris'

    iris = Dataset.get_by_name(workspace= ws, name=dataset_name) 

    #load a tabularDataset by name
    iris_df = iris.to_pandas_dataframe()

    # X -> features, y -> label
   
    X = iris_df.iloc[:,:3]
    y = iris_df.iloc[:,4]

    # dividing X, y into train and test data
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

    # training a linear SVM classifier
    from sklearn.svm import SVC
    svm_model_linear = SVC(kernel=args.kernel, C=args.penalty).fit(X_train, y_train)
    svm_predictions = svm_model_linear.predict(X_test)

    # model accuracy for X_test
    accuracy = svm_model_linear.score(X_test, y_test)
    print('Accuracy of SVM classifier on test set: {:.2f}'.format(accuracy))
    run.log('Accuracy', np.float(accuracy))
    # creating a confusion matrix
    cm = confusion_matrix(y_test, svm_predictions)
    print(cm)

    os.makedirs('outputs', exist_ok=True)
    # files saved in the "outputs" folder are automatically uploaded into run history
    joblib.dump(svm_model_linear, 'outputs/model.joblib')


if __name__ == '__main__':
    main()


In [12]:
from azureml.core import Workspace, Run
ws = Workspace.from_config()
import argparse
import os
from azureml.core import Dataset
# importing necessary libraries
import numpy as np

from sklearn import datasets
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

import joblib

dataset_name = 'iris'

iris = Dataset.get_by_name(workspace= ws, name=dataset_name) 

#load a tabularDataset by name
iris_df = iris.to_pandas_dataframe()

# X -> features, y -> label
   
X = iris_df.iloc[:,:3]
y = iris_df.iloc[:,4]

# dividing X, y into train and test data
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

# training a linear SVM classifier
from sklearn.svm import SVC
svm_model_linear = SVC(kernel='linear', C=0.01).fit(X_train, y_train)



ValueError: Input contains NaN, infinity or a value too large for dtype('float64').